## Interpreter Settings

In [1]:
import sys
print(sys.version)
print(sys.version_info)
print(sys.api_version)
print(sys.hexversion)

3.6.6 (default, Sep 12 2018, 18:26:19) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]
sys.version_info(major=3, minor=6, micro=6, releaselevel='final', serial=0)
1013
50726640


In [2]:
print(sys.platform)

darwin


In [4]:
print(sys.implementation.name)
print(sys.implementation.version)
print(sys.implementation.cache_tag)

cpython
sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)
cpython-36


CPython Command Line Option Flags  

|Option	|Meaning|
|------|------|
|-B	|do not write .py[co] files on import|
|-b	|issue warnings about converting bytes to string without decoding properly and comparing bytes with strings|  
|-bb	|convert bytes warnings to errors|
|-d	|debug output from parser|
|-E	|ignore PYTHON* environment variables (such as PYTHONPATH)|
|-i	|inspect interactively after running script|
|-O	|optimize generated bytecode slightly|
|-OO	|remove doc-strings in addition to the -O optimizations|
|-s	|do not add user site directory to sys.path|
|-S	|do not run ‘import site’ on initialization|
|-t	|issue warnings about inconsistent tab usage|
|-tt	|issue errors for inconsistent tab usage|
|-v	|verbose|

In [5]:
if sys.flags.bytes_warning:
    print('Warning on bytes/str errors')
if sys.flags.debug:
    print('Debuging')
if sys.flags.inspect:
    print('Will enter interactive mode after running')
if sys.flags.optimize:
    print('Optimizing byte-code')
if sys.flags.dont_write_bytecode:
    print('Not writing byte-code files')
if sys.flags.no_site:
    print('Not importing "site"')
if sys.flags.ignore_environment:
    print('Ignoring environment')
if sys.flags.verbose:
    print('Verbose mode')

```
$ python3 -S -E -b sys_flags.py

Warning on bytes/str errors
Not importing "site"
Ignoring environment
```

In [7]:
print(sys.getfilesystemencoding())
print(sys.getdefaultencoding())

utf-8
utf-8


In [9]:
print(sys.ps1)
print(sys.ps2)

In : 
...: 


In [10]:
class LineCounter:

    def __init__(self):
        self.count = 0

    def __str__(self):
        self.count += 1
        return '({:3d})> '.format(self.count)

```
$ python
Python 3.4.2 (v3.4.2:ab2c023a9432, Oct  5 2014, 20:42:22)
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)] on darwin
Type "help", "copyright", "credits" or "license" for more
information.
>>> from sys_ps1 import LineCounter
>>> import sys
>>> sys.ps1 = LineCounter()
(  1)>
(  2)>
(  3)>
```

In [11]:
print(sys.executable)
print(sys.prefix)

/usr/local/opt/python3/bin/python3.6
/usr/local/Cellar/python3/3.6.4_1/Frameworks/Python.framework/Versions/3.6


## Runtime Environment

In [12]:
# sys_argv.py
print('Arguments:', sys.argv)

Arguments: ['/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py', '-f', '/Users/hejl/Library/Jupyter/runtime/kernel-016356e5-8850-49d4-a189-6e7ae991a82c.json']


```
$ python3 sys_argv.py

Arguments: ['sys_argv.py']

$ python3 sys_argv.py -v foo blah

Arguments: ['sys_argv.py', '-v', 'foo', 'blah']

$ python3 -u sys_argv.py   // the -u option is understood by the interpreter, and is not passed to the program being run

Arguments: ['sys_argv.py']
```

In [2]:
# sys_stdio.py

print('STATUS: Reading from stdin', file=sys.stderr)

data = sys.stdin.read()

print('STATUS: Writing data to stdout', file=sys.stderr)

sys.stdout.write(data)
sys.stdout.flush()

print('STATUS: Done', file=sys.stderr)

STATUS: Reading from stdin
STATUS: Writing data to stdout
STATUS: Done


```
$ cat sys_stdio.py | python3 -u sys_stdio.py

STATUS: Reading from stdin
STATUS: Writing data to stdout
#!/usr/bin/env python3

#end_pymotw_header
import sys

print('STATUS: Reading from stdin', file=sys.stderr)

data = sys.stdin.read()

print('STATUS: Writing data to stdout', file=sys.stderr)

sys.stdout.write(data)
sys.stdout.flush()

print('STATUS: Done', file=sys.stderr)
STATUS: Done
```

In [3]:
# sys_exit.py
exit_code = int(sys.argv[1])
sys.exit(exit_code)

ValueError: invalid literal for int() with base 10: '-f'

```
$ python3 sys_exit.py 0 ; echo "Exited $?"

Exited 0

$ python3 sys_exit.py 1 ; echo "Exited $?"

Exited 1

```

## Memory Management and Limits

In [4]:
one = []
print(sys.getrefcount(one))
two = one
print(sys.getrefcount(one))
del two 
print(sys.getrefcount(one))

2
3
2


In [5]:
class Myclass:
    pass

objects = [[],(),{},set(),'c','c1234',b'bytes',1,2.3,Myclass, Myclass()]

for obj in objects:
    print(f"{type(obj).__name__} : {sys.getsizeof(obj)}")

list : 64
tuple : 48
dict : 240
set : 224
str : 50
str : 54
bytes : 38
int : 28
float : 24
type : 1056
Myclass : 56


In [8]:
class WithoutAttr:
    pass
class WithAttr:
    def __init__(self,a,b):
        self.a = a
        self.b = b
        return
    
print(sys.getsizeof(WithoutAttr()))
print(sys.getsizeof(WithAttr('a33','b33')))

56
56


In [11]:
class WithoutAttr:
    def __sizeof__(self):
        return object.__sizeof__(self) + sum(sys.getsizeof(v) for v in self.__dict__.values())
class WithAttr:
    def __init__(self,a,b):
        self.a = a
        self.b = b
        return
    def __sizeof__(self):
        return object.__sizeof__(self) + sum(sys.getsizeof(v) for v in self.__dict__.values())
print(sys.getsizeof(WithoutAttr()))
print(sys.getsizeof(WithAttr('a33','b33')))

56
160


In [18]:
print('init recur limit:',sys.getrecursionlimit())
sys.setrecursionlimit(55)
print('modified:',sys.getrecursionlimit())
def gen_recur_error(i):
    print('gen_recur_error:',i)
    gen_recur_error(i+1)
try:
    gen_recur_error(1)
except RecursionError as err:
    print(err)

init recur limit: 100
modified: 55
gen_recur_error: 1
gen_recur_error: 2
gen_recur_error: 3
gen_recur_error: 4
gen_recur_error: 5
gen_recur_error: 6
gen_recur_error: 7
gen_recur_error: 8
gen_recur_error: 9
gen_recur_error: 10
gen_recur_error:maximum recursion depth exceeded in comparison


In [19]:
print(sys.maxsize)
print(sys.maxunicode)

9223372036854775807
1114111


maxsize is the maximum size of a list, dictionary, string, or other data structure dictated by the C interpreter’s size type. maxunicode is the largest integer Unicode point supported by the interpreter as currently configured.

In [20]:
print('Smallest difference (epsilon):', sys.float_info.epsilon)
print()
print('Digits (dig)              :', sys.float_info.dig)
print('Mantissa digits (mant_dig):', sys.float_info.mant_dig)
print()
print('Maximum (max):', sys.float_info.max)
print('Minimum (min):', sys.float_info.min)
print()
print('Radix of exponents (radix):', sys.float_info.radix)
print()
print('Maximum exponent for radix (max_exp):', sys.float_info.max_exp)
print('Minimum exponent for radix (min_exp):', sys.float_info.min_exp)
print()
print('Max. exponent power of 10 (max_10_exp):', sys.float_info.max_10_exp)
print('Min. exponent power of 10 (min_10_exp):', sys.float_info.min_10_exp)
print()
print('Rounding for addition (rounds):', sys.float_info.rounds)

Smallest difference (epsilon): 2.220446049250313e-16

Digits (dig)              : 15
Mantissa digits (mant_dig): 53

Maximum (max): 1.7976931348623157e+308
Minimum (min): 2.2250738585072014e-308

Radix of exponents (radix): 2

Maximum exponent for radix (max_exp): 1024
Minimum exponent for radix (min_exp): -1021

Max. exponent power of 10 (max_10_exp): 308
Min. exponent power of 10 (min_10_exp): -307

Rounding for addition (rounds): 1


In [21]:
print('Number of bits used to hold each digit:',sys.int_info.bits_per_digit)
print('Size in bytes of C type used to hold each digit:',sys.int_info.sizeof_digit)

Number of bits used to hold each digit: 30
Size in bytes of C type used to hold each digit: 4


In [22]:
print(sys.byteorder)

little


## Exception Handling

In [2]:
import sys
def my_excepthook(type, value, trackback):
    print('Unhandled error:', type, value)
    
sys.excepthook = my_excepthook
raise RuntimeError("this is the error message")

RuntimeError: this is the error message

In [3]:
import threading
import time
def do_sth_with_exception():
    exc_type, exc_value = sys.exc_info()[:2]
    print(f"handling {exc_type.__name__} exception with message {exc_value} in {threading.current_thread().name}")
    
def cause_exception(delay):
    time.sleep(delay)
    raise RuntimeError("This is the error msg")
    
def thread_target(delay):
    try:
        cause_exception(delay)
    except RuntimeError:
        do_sth_with_exception()
        
threads = [threading.Thread(target=thread_target, args=(1,)),threading.Thread(target=thread_target, args=(2,))]
for t in threads:
    t.start()
for t in threads:
    t.join()

handling RuntimeError exception with message This is the error msg in Thread-4
handling RuntimeError exception with message This is the error msg in Thread-5


## Low-level Thread Support

In [2]:
import sys
import threading
from queue import Queue

def show_thread(q):
    for i in range(5):
        for j in range(100000):
            pass
        q.put(threading.current_thread().name)
    return

def run_threads():
    interval = sys.getswitchinterval()
    print("interval = ", interval)
    q = Queue()
    threads = [threading.Thread(target=show_thread, args=(q,), name=f"T{i}") for i in range(3)]
    for t in threads:
        t.setDaemon(True)
        t.start()
    for t in threads:
        t.join()
    while not q.empty():
        print(q.get(), end="  ")
    print()
    return

In [5]:
for interval in [0.0001,0.001,0.01, 0.1]:
    sys.setswitchinterval(interval)
    run_threads()
    print()

interval =  9.999999999999999e-05
T0  T1  T2  T0  T1  T2  T0  T1  T2  T0  T2  T2  T1  T0  T1  

interval =  0.001
T0  T0  T1  T2  T0  T1  T2  T0  T1  T2  T0  T2  T1  T1  T2  

interval =  0.01
T0  T0  T0  T0  T0  T1  T1  T1  T1  T1  T2  T2  T2  T2  T2  

interval =  0.09999999999999999
T0  T0  T0  T0  T0  T1  T1  T1  T1  T1  T2  T2  T2  T2  T2  



In [6]:
import time
io_lock = threading.Lock()
blocker = threading.Lock()

def block(i):
    t = threading.current_thread()
    with io_lock:
        print(f"{t.name} with ident {t.ident} going to sleep")
    if i:
        blocker.acquire()  # acquired but never resleased
        time.sleep(0.2)
    with io_lock:
        print(t.name, 'finishing')

threads = [threading.Thread(target=block, args=(i,)) for i in range(4)]
for t in threads:
    t.setDaemon(True)
    t.start()
    
threads_by_ident = dict((t.ident, t) for t in threads)

# show where each thread is "blocked"
time.sleep(0.01)
with io_lock:
    for ident, frame in sys._current_frames().items():
        t = threads_by_ident.get(ident)
        if not t:
            # main thread
            continue
        print(f"{t.name} stopped in {frame.f_code.co_name} at line {frame.f_lineno} of {frame.f_code.co_filename}")

Thread-4 with ident 123145554059264 going to sleep
Thread-4 finishing
Thread-5 with ident 123145554059264 going to sleep
Thread-6 with ident 123145559314432 going to sleep
Thread-7 with ident 123145564569600 going to sleep
Thread-7 stopped in block at line 10 of <ipython-input-6-2161a65be775>
Thread-6 stopped in block at line 10 of <ipython-input-6-2161a65be775>
Thread-5 stopped in block at line 11 of <ipython-input-6-2161a65be775>
Thread-5 finishing


## Modules and Imports

In [1]:
import sys
print(sys.modules)

{'builtins': <module 'builtins' (built-in)>, 'sys': <module 'sys' (built-in)>, '_frozen_importlib': <module 'importlib._bootstrap' (frozen)>, '_imp': <module '_imp' (built-in)>, '_warnings': <module '_warnings' (built-in)>, '_thread': <module '_thread' (built-in)>, '_weakref': <module '_weakref' (built-in)>, '_frozen_importlib_external': <module 'importlib._bootstrap_external' (frozen)>, '_io': <module 'io' (built-in)>, 'marshal': <module 'marshal' (built-in)>, 'posix': <module 'posix' (built-in)>, 'zipimport': <module 'zipimport' (built-in)>, 'encodings': <module 'encodings' from '/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/encodings/__init__.py'>, 'codecs': <module 'codecs' from '/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/codecs.py'>, '_codecs': <module '_codecs' (built-in)>, 'encodings.aliases': <module 'encodings.aliases' from '/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versio

In [4]:
print(sys.modules.keys())

dict_keys(['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', '_sysconfigdata_m_darwin_darwin', '_osx_support', 're', 'enum', 'types', 'functools', '_functools', 'collections', 'operator', '_operator', 'keyword', 'heapq', '_heapq', 'itertools', 'reprlib', '_collections', 'weakref', 'collections.abc', 'sre_compile', '_sre', 'sre_parse', 'sre_constants', 'copyreg', 'importlib', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib.util', 'importlib.abc', 'importlib.machinery', 'contextlib', 'google', 'google.cloud', 'google.cloud.gapic', 'google.cloud.gapic.datastor

In [5]:
print(sys.builtin_module_names)

('_ast', '_codecs', '_collections', '_functools', '_imp', '_io', '_locale', '_operator', '_signal', '_sre', '_stat', '_string', '_symtable', '_thread', '_tracemalloc', '_warnings', '_weakref', 'atexit', 'builtins', 'errno', 'faulthandler', 'gc', 'itertools', 'marshal', 'posix', 'pwd', 'sys', 'time', 'xxsubtype', 'zipimport')


In [6]:
print(sys.path)

['', '/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python36.zip', '/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6', '/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/site-packages/pip_pop-0.1.0-py3.6.egg', '/usr/local/lib/python3.6/site-packages/pip_grep-0.0.0-py3.6.egg', '/usr/local/lib/python3.6/site-packages/lp-0.3-py3.6.egg', '/usr/local/lib/python3.6/site-packages/Show_py_program-0.3-py3.6.egg', '/usr/local/lib/python3.6/site-packages/Show_py_program_description-0.1.0-py3.6.egg', '/usr/local/lib/python3.6/site-packages/IPython/extensions', '/Users/hejl/.ipython']


In [7]:
sys.path.insert(0, '.')
print(sys.path)

['.', '', '/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python36.zip', '/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6', '/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/site-packages/pip_pop-0.1.0-py3.6.egg', '/usr/local/lib/python3.6/site-packages/pip_grep-0.0.0-py3.6.egg', '/usr/local/lib/python3.6/site-packages/lp-0.3-py3.6.egg', '/usr/local/lib/python3.6/site-packages/Show_py_program-0.3-py3.6.egg', '/usr/local/lib/python3.6/site-packages/Show_py_program_description-0.1.0-py3.6.egg', '/usr/local/lib/python3.6/site-packages/IPython/extensions', '/Users/hejl/.ipython']


In [9]:
print(sys.path_hooks)

[<class 'zipimport.zipimporter'>, <function FileFinder.path_hook.<locals>.path_hook_for_FileFinder at 0x1093de7b8>]


In [11]:
import shelve
import os

filename = '/tmp/pymotw_import_example.shelve'
if os.path.exists(filename + '.db'):
    os.unlink(filename + '.db')
with shelve.open(filename) as db:
    db['data:README'] = b"""
==============
package README
==============

This is the README for ``package``.
"""
    db['package.__init__'] = b"""
print('package imported')
message = 'This message is in package.__init__'
"""
    db['package.module1'] = b"""
print('package.module1 imported')
message = 'This message is in package.module1'
"""
    db['package.subpackage.__init__'] = b"""
print('package.subpackage imported')
message = 'This message is in package.subpackage.__init__'
"""
    db['package.subpackage.module2'] = b"""
print('package.subpackage.module2 imported')
message = 'This message is in package.subpackage.module2'
"""
    db['package.with_error'] = b"""
print('package.with_error being imported')
raise ValueError('raising exception to break import')
"""
    print('Created {} with:'.format(filename))
    for key in sorted(db.keys()):
        print('  ', key)

Created /tmp/pymotw_import_example.shelve with:
   data:README
   package.__init__
   package.module1
   package.subpackage.__init__
   package.subpackage.module2
   package.with_error


In [13]:
import sys_shelve_importer
def show_module_details(module):
    print("message:" , module.message)
    print("__name__:", module.__name__)
    print("__package__:", module.__package__)
    print("__file__:", module.__file__)
    print("__path__:", module.__path__)
    print("__loader__:", module.__loader__)
    
sys.path_hooks.append(sys_shelve_importer.ShelveFinder)
sys.path.insert(0, filename)

In [14]:
print('import package')
import package

import package
shelf add to import path: /tmp/pymotw_import_example.shelve
looking for 'package' in /tmp/pymotw_import_example.shelve
found it as package.__init__
loading source for package from shelf
create a new module obj for package
adding path to package
execing source...
package imported
done


In [15]:
show_module_details(package)

message: This message is in package.__init__
__name__: package
__package__: package
__file__: /tmp/pymotw_import_example.shelve/package
__path__: ['/tmp/pymotw_import_example.shelve']
__loader__: <sys_shelve_importer.ShelveLoader object at 0x10b4dce10>


In [17]:
package.__dict__

{'__builtins__': {'ArithmeticError': ArithmeticError,
  'AssertionError': AssertionError,
  'AttributeError': AttributeError,
  'BaseException': BaseException,
  'BlockingIOError': BlockingIOError,
  'BrokenPipeError': BrokenPipeError,
  'BufferError': BufferError,
  'BytesWarning': BytesWarning,
  'ChildProcessError': ChildProcessError,
  'ConnectionAbortedError': ConnectionAbortedError,
  'ConnectionError': ConnectionError,
  'ConnectionRefusedError': ConnectionRefusedError,
  'ConnectionResetError': ConnectionResetError,
  'DeprecationWarning': DeprecationWarning,
  'EOFError': EOFError,
  'Ellipsis': Ellipsis,
  'EnvironmentError': OSError,
  'Exception': Exception,
  'False': False,
  'FileExistsError': FileExistsError,
  'FileNotFoundError': FileNotFoundError,
  'FloatingPointError': FloatingPointError,
  'FutureWarning': FutureWarning,
  'GeneratorExit': GeneratorExit,
  'IOError': OSError,
  'ImportError': ImportError,
  'ImportWarning': ImportWarning,
  'IndentationError': Ind

In [19]:
print(sys.modules['package'])

<module 'package' (<sys_shelve_importer.ShelveLoader object at 0x10b4dce10>)>


In [20]:
import package.module1

looking for 'package.module1' in /tmp/pymotw_import_example.shelve
found it as package.module1
loading source for package.module1 from shelf
create a new module obj for package.module1
import as regular module
execing source...
package.module1 imported
done


In [21]:
show_module_details(package.module1)

message: This message is in package.module1
__name__: package.module1
__package__: package
__file__: /tmp/pymotw_import_example.shelve/package.module1
__path__: /tmp/pymotw_import_example.shelve
__loader__: <sys_shelve_importer.ShelveLoader object at 0x10b53aba8>


In [23]:
show_module_details(package.subpackage.module2)

AttributeError: module 'package' has no attribute 'subpackage'

In [24]:
import package.subpackage.module2

looking for 'package.subpackage' in /tmp/pymotw_import_example.shelve
found it as package.subpackage.__init__
loading source for package.subpackage from shelf
create a new module obj for package.subpackage
adding path to package
execing source...
package.subpackage imported
done
looking for 'package.subpackage.module2' in /tmp/pymotw_import_example.shelve
found it as package.subpackage.module2
loading source for package.subpackage.module2 from shelf
create a new module obj for package.subpackage.module2
import as regular module
execing source...
package.subpackage.module2 imported
done


In [25]:
show_module_details(package.subpackage.module2)

message: This message is in package.subpackage.module2
__name__: package.subpackage.module2
__package__: package.subpackage
__file__: /tmp/pymotw_import_example.shelve/package.subpackage.module2
__path__: /tmp/pymotw_import_example.shelve
__loader__: <sys_shelve_importer.ShelveLoader object at 0x10b518c88>


In [26]:
import package

In [27]:
import importlib
importlib.reload(package)

looking for 'package' in /tmp/pymotw_import_example.shelve
found it as package.__init__
loading source for package from shelf
reusing module from import of package
adding path to package
execing source...
package imported
done


<module 'package' (<sys_shelve_importer.ShelveLoader object at 0x10b522208>)>

In [28]:
try:
    import aaa
except ImportError as e:
    print("failed to import ", e)

looking for 'aaa' in /tmp/pymotw_import_example.shelve
not found
failed to import  No module named 'aaa'


In [30]:
sys.path_hooks.pop(2)

sys_shelve_importer.ShelveFinder

In [32]:
sys.path_hooks

[zipimport.zipimporter,
 <function _frozen_importlib_external.FileFinder.path_hook.<locals>.path_hook_for_FileFinder>]

In [33]:
try:
    import aaa
except ImportError as e:
    print("failed to import ", e)

looking for 'aaa' in /tmp/pymotw_import_example.shelve
not found
failed to import  No module named 'aaa'


In [34]:
import pkgutil
readme_path = os.path.join(package.__path__[0], 'README')
readme = pkgutil.get_data('package', 'README') # equal to package.__loader__.get_data(readme_path)
print(readme.decode('utf-8'))

foo_path = os.path.join(package.__path__[0], 'foo')
try:
    foo = pkgutil.get_data('package', 'foo') 
except IOError as e:
    print('could not load foo', e)
else:
    print(foo)

looking for data in /tmp/pymotw_import_example.shelve for /tmp/pymotw_import_example.shelve/README

package README

This is the README for ``package``.

looking for data in /tmp/pymotw_import_example.shelve for /tmp/pymotw_import_example.shelve/foo
could not load foo 


In [35]:
prefix = os.path.abspath(sys.prefix)
print(prefix)
for name in sys.path:
    name = name.replace(prefix, '...')
    print('  ', name)

/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6
   /tmp/pymotw_import_example.shelve
   .
   
   .../lib/python36.zip
   .../lib/python3.6
   .../lib/python3.6/lib-dynload
   /usr/local/lib/python3.6/site-packages
   /usr/local/lib/python3.6/site-packages/pip_pop-0.1.0-py3.6.egg
   /usr/local/lib/python3.6/site-packages/pip_grep-0.0.0-py3.6.egg
   /usr/local/lib/python3.6/site-packages/lp-0.3-py3.6.egg
   /usr/local/lib/python3.6/site-packages/Show_py_program-0.3-py3.6.egg
   /usr/local/lib/python3.6/site-packages/Show_py_program_description-0.1.0-py3.6.egg
   /usr/local/lib/python3.6/site-packages/IPython/extensions
   /Users/hejl/.ipython


In [36]:
for name, cache_value in sys.path_importer_cache.items():
    if '..' in name:
        name = os.path.abspath(name)
    name = name.replace(prefix, '...')
    print(f"  {name}:{cache_value!r}")

  .../lib/python36.zip:None
  .../lib/python3.6:FileFinder('/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6')
  .../lib/python3.6/encodings:FileFinder('/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/encodings')
  .../lib/python3.6/lib-dynload:FileFinder('/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/lib-dynload')
  .../lib/python3.6/collections:FileFinder('/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/collections')
  .../lib/python3.6/importlib:FileFinder('/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib')
  .../lib/python3.6/site-packages:FileFinder('/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages')
  .../lib/python3.6/site-packages/google:FileFinder('/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3

In [37]:
sys.path_importer_cache

{'.': FileFinder('.'),
 '/Users/hejl/.ipython': FileFinder('/Users/hejl/.ipython'),
 '/Users/hejl/.ipython/extensions': FileFinder('/Users/hejl/.ipython/extensions'),
 '/Users/hejl/JianGuo/practise/standard_library/RuntimeFeature': FileFinder('/Users/hejl/JianGuo/practise/standard_library/RuntimeFeature'),
 '/tmp/pymotw_import_example.shelve': <sys_shelve_importer.ShelveFinder at 0x10b4dc8d0>,
 '/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6': FileFinder('/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6'),
 '/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/collections': FileFinder('/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/collections'),
 '/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/concurrent': FileFinder('/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/

In [38]:
sys.meta_path

[_frozen_importlib.BuiltinImporter,
 _frozen_importlib.FrozenImporter,
 _frozen_importlib_external.PathFinder,

In [1]:
import imp
import sys
class NoisyMetaImportFinder:
    def __init__(self, prefix):
        print(f"creating NoisyMetaImportFinder for {prefix}")
        self.prefix = prefix
    def find_module(self, fullname, path=None):
        print(f"looking for {fullname !r} with path {path !r}")
        name_parts = fullname.split('.')
        if name_parts and name_parts[0] == self.prefix:
            print('... found prefix, return loader')
            return NoisyMetaImportLoader(path)
        else:
            print('not right prefix')
            
class NoisyMetaImportLoader:
    def __init__(self, path_entry):
        self.path_entry = path_entry
    
    def load_module(self, fullname):
        print(f"loading {fullname}")
        if fullname in sys.modules:
            mod = sys.modules[fullname]
        else:
            mod = sys.modules.setdefault(fullname, imp.new_module(fullname))
            
        mod.__file__ = fullname
        mod.__name__ = fullname
        mod.__path__ = ['path-entry-goes-here']
        mod.__loader__ = self
        mod.__package__ = '.'.join(fullname.split('.')[:-1])
        return mod

In [2]:
sys.meta_path.append(NoisyMetaImportFinder('foo'))

import foo

creating NoisyMetaImportFinder for foo
looking for 'foo' with path None
... found prefix, return loader
loading foo


In [3]:
import foo.bar

looking for 'foo.bar' with path ['path-entry-goes-here']
... found prefix, return loader
loading foo.bar


In [4]:
try:
    import bar
except:
    pass

looking for 'bar' with path None
not right prefix


## Tracing a Program As It Runs

|Event	|When it occurs	|Argument value| 
| ----- | ------------- | ----------|
|call	|Before a line is executed|	None|
|line	|Before a line is executed|	None
|return	|Before a function returns|	The value being returned
|exception	|After an exception occurs|	The (exception, value, traceback) tuple
|c_call	|Before a C function is called|	The C function object
|c_return	|After a C function returns|	None
|c_exception	|After a C function throws an error|	None

In [1]:
import sys

TRACE_INFO = ['a','b','c']
def trace_exec(frame, event, arg):
    if event != 'exception':
        return
    co = frame.f_code
    func_name = co.co_name
    line_no = frame.f_lineno
    exc_type, exc_value, exc_traceback = arg
    print(f"* Tracing exception: \n {exc_type.__name__} {exc_value} \n on line {line_no} of {func_name}")
    
def trace_calls(frame, event, arg):
    if event != 'call':
        return
    co = frame.f_code
    func_name = co.co_name
    if func_name in TRACE_INFO:
        return trace_exec


def c():
    raise RuntimeError('generate exception in c()')


def b():
    c()
    print('leaving b() \n')


def a():
    b()
    print('leaving a() \n')


sys.settrace(trace_calls)
try:
    a()
except Exception as e:
    print('exception handler :', e)


* Tracing exception: 
 RuntimeError generate exception in c() 
 on line 23 of c
* Tracing exception: 
 RuntimeError generate exception in c() 
 on line 27 of b
* Tracing exception: 
 RuntimeError generate exception in c() 
 on line 32 of a
exception handler : generate exception in c()
